In [2]:
%pip install faiss-cpu
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = WebBaseLoader("https://lilianweng.github.io/posts/2023-06-23-agent/")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

vectorstore = FAISS.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

llm = ChatOpenAI()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
from langchain import hub
from langchain.chains import RetrievalQA

LANGCHAIN_API_KEY = os.getenv('LANGCHAIN_API_KEY')
LANGCHAIN_API_KEY


'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'

In [6]:
%whos

Variable                         Type                              Data/Info
----------------------------------------------------------------------------
ChatOpenAI                       ModelMetaclass                    <class 'langchain_openai.<...>_models.base.ChatOpenAI'>
FAISS                            ABCMeta                           <class 'langchain_communi<...>ectorstores.faiss.FAISS'>
LANGCHAIN_API_KEY                str                               lsv2_pt_c235bcb425d144729<...>4d14df5abaf7a8_ab68fc6636
OpenAIEmbeddings                 ModelMetaclass                    <class 'langchain_openai.<...>s.base.OpenAIEmbeddings'>
RecursiveCharacterTextSplitter   ABCMeta                           <class 'langchain_text_sp<...>veCharacterTextSplitter'>
RetrievalQA                      ModelMetaclass                    <class 'langchain.chains.<...>val_qa.base.RetrievalQA'>
WebBaseLoader                    ABCMeta                           <class 'langchain_communi<...>.web_base.W

In [7]:
prompt = hub.pull("rlm/rag-prompt")

qa_chain = RetrievalQA.from_llm(
    llm, ## llm 모델 설정 
    retriever = vectorstore.as_retriever(), ## vector store 에서 문서 검색
    prompt= prompt
)

qa_chain("What are autonomous agents?")

C:\Users\1\AppData\Local\Temp\ipykernel_14780\1693774383.py:9: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  qa_chain("What are autonomous agents?")


{'query': 'What are autonomous agents?',
 'result': 'Autonomous agents are LLM-powered agents that can handle the autonomous design, planning, and performance of complex scientific experiments. These agents can browse the Internet, read documentation, execute code, call robotics experimentation APIs, and leverage other LLMs. They can develop novel solutions by reasoning through different steps and accessing external APIs for additional information.'}

In [16]:
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

qa_chain = (
    {
            "context": vectorstore.as_retriever() | format_docs,
            "question": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
    )

qa_chain.invoke("What are autonomous agents?")

'Autonomous agents are AI systems powered by LLMs that can handle complex scientific experiments by autonomously designing, planning, and performing tasks. These agents can browse the Internet, read documentation, execute code, call robotics experimentation APIs, and leverage other LLMs to gather missing information and improve their reasoning abilities. These agents can be used for various tasks, such as developing novel anticancer drugs, by utilizing external APIs for extra information and access to proprietary sources.'